In [1]:
!pip install -q keras-nlp --upgrade

In [2]:
!pip install gdown;
!gdown --id 1pVx4zlZEnpeU5ioAifpQ6PdR1WSJ_dkA;
!gdown --id 1T5lZ4AeaCo-puDxDR3CANeZyG7MEmN3I;

/opt/conda/lib/python3.10/site-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1pVx4zlZEnpeU5ioAifpQ6PdR1WSJ_dkA
From (redirected): https://drive.google.com/uc?id=1pVx4zlZEnpeU5ioAifpQ6PdR1WSJ_dkA&confirm=t&uuid=56f927af-fe79-4e37-bcdb-e6219626bf51
To: /kaggle/working/ELECTRA_EN.weights.h5
100%|█████████████████████████████████████████| 272M/272M [00:02<00:00, 123MB/s]
/opt/conda/lib/python3.10/site-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1T5lZ4AeaCo-puDxDR3CANeZyG7MEmN3I
From (redirected): https://drive.google.com/uc?id=1T5lZ4AeaCo-puDxDR3CANeZyG7MEmN3I&confirm=t&uuid=a67154

In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import keras_nlp
import keras
from kerastuner import HyperModel
from kerastuner.tuners import RandomSearch
import tensorflow as tf
import os

keras.mixed_precision.set_global_policy("mixed_float16")

label_names = ["entailment", "neutral", "contradiction"]

/tmp/ipykernel_23/4082429652.py:5: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import HyperModel


In [4]:
DATA_DIR = '/kaggle/input/contradictory-my-dear-watson/'
for dirname, _, filenames in os.walk(DATA_DIR):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/contradictory-my-dear-watson/sample_submission.csv
/kaggle/input/contradictory-my-dear-watson/train.csv
/kaggle/input/contradictory-my-dear-watson/test.csv


In [5]:
df_train = pd.read_csv(DATA_DIR + "train.csv")
df_train.head()

,id,premise,hypothesis,lang_abv,language,label
0,5130fd2cb5,and these comments were considered in formulat...,The rules developed in the interim were put to...,en,English,0
1,5b72532a0b,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,en,English,2
2,3931fbe82a,Des petites choses comme celles-là font une di...,J'essayais d'accomplir quelque chose.,fr,French,0
3,5622f0c60b,you know they can't really defend themselves l...,They can't defend themselves because of their ...,en,English,0
4,86aaa48b45,ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...,เด็กสามารถเห็นได้ว่าชาติพันธุ์แตกต่างกันอย่างไร,th,Thai,1


In [6]:
def split_labels(x, y):
    return (x[0], x[1]), y

In [7]:
batch_size = 28
buffer_size = batch_size * 10

training_dataset = (
    tf.data.Dataset.from_tensor_slices(
        (
            df_train[["hypothesis", "premise"]].values,
            df_train["label"].values
        )
    )
)

print(f"Size of the train dataset: {len(training_dataset)}")

train_preprocessed = training_dataset.shuffle(buffer_size=buffer_size).map(split_labels, tf.data.AUTOTUNE).batch(batch_size, drop_remainder=True).prefetch(tf.data.AUTOTUNE)

Size of the train dataset: 12120


In [8]:
# List both files and directories
entries = os.listdir("/kaggle/working")

for entry in entries:
    print(entry)


Tuned_ELECTRA_EN.weights.h5
ELECTRA_EN.weights.h5
__notebook__.ipynb


In [ ]:
electra_discriminator_token = keras_nlp.models.Tokenizer.from_preset("electra_small_discriminator_uncased_en")
bert_preprocessor = keras_nlp.models.BertTextClassifierPreprocessor(electra_discriminator_token, sequence_length=240)

train_set = (
    train_preprocessed.map(bert_preprocessor, tf.data.AUTOTUNE).cache().prefetch(tf.data.AUTOTUNE)
)

electra_discriminator_back = keras_nlp.models.Backbone.from_preset("electra_small_discriminator_uncased_en")

electra_classifier = keras_nlp.models.BertClassifier(electra_discriminator_back, 3, preprocessor=None)

electra_classifier.load_weights("/kaggle/working/Tuned_ELECTRA_EN.weights.h5")

/opt/conda/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'loss_scale_optimizer', because it has 4 variables whereas the saved optimizer has 412 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/opt/conda/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 408 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [10]:
df_test = pd.read_csv(DATA_DIR + "test.csv")
print(df_test.head())
df_test["label"] = [0] * len(df_test)

           id                                            premise  \
0  c6d58c3f69  بکس، کیسی، راہیل، یسعیاہ، کیلی، کیلی، اور کولم...   
1  cefcc82292                             هذا هو ما تم نصحنا به.   
2  e98005252c  et cela est en grande partie dû au fait que le...   
3  58518c10ba                   与城市及其他公民及社区组织代表就IMA的艺术发展进行对话&amp   
4  c32b0d16df                              Она все еще была там.   

                                          hypothesis lang_abv language  
0  کیسی کے لئے کوئی یادگار نہیں ہوگا, کولمین ہائی...       ur     Urdu  
1  عندما يتم إخبارهم بما يجب عليهم فعله ، فشلت ال...       ar   Arabic  
2                             Les mères se droguent.       fr   French  
3                            IMA与其他组织合作，因为它们都依靠共享资金。       zh  Chinese  
4     Мы думали, что она ушла, однако, она осталась.       ru  Russian  


In [11]:
testing_dataset = (
    tf.data.Dataset.from_tensor_slices(
        (
            df_test[["hypothesis", "premise"]].values,
            df_test["label"].values
        )
    )
)

print(len(testing_dataset))

test_preprocessed = testing_dataset.map(split_labels, tf.data.AUTOTUNE).batch(1, drop_remainder=True).cache().prefetch(tf.data.AUTOTUNE)
bert_test_set = (test_preprocessed.map(bert_preprocessor, tf.data.AUTOTUNE).cache().prefetch(tf.data.AUTOTUNE))

5195


In [12]:
predictions = electra_classifier.predict(bert_test_set)

I0000 00:00:1733095403.648227      75 service.cc:145] XLA service 0x7f9860002cb0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1733095403.648298      75 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0


  34/5195 ━━━━━━━━━━━━━━━━━━━━ 24s 5ms/step

I0000 00:00:1733095406.135219      75 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


5195/5195 ━━━━━━━━━━━━━━━━━━━━ 26s 4ms/step


In [13]:
predicted_classes = tf.argmax(predictions, axis=1)
predicted_classes_np = predicted_classes.numpy()
print(predicted_classes_np)

[0 0 0 ... 0 0 2]


In [14]:
submission = df_test.id.copy().to_frame()
submission["prediction"] = predicted_classes_np
submission.to_csv("submission.csv", index=False)

submission

,id,prediction
0,c6d58c3f69,0
1,cefcc82292,0
2,e98005252c,0
3,58518c10ba,0
4,c32b0d16df,0
...,...,...
5190,5f90dd59b0,0
5191,f357a04e86,1
5192,1f0ea92118,0
5193,0407b48afb,0
